<a href="https://www.kaggle.com/code/aliabdelmenam/used-car-price-regreesion?scriptVersionId=195698299" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/aliabdelmonam/Used-Car-Price-for-Regression/blob/main/Regreesion_for_used_car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from datetime import date
from sklearn.ensemble import IsolationForest,VotingRegressor

In [2]:
tr= pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
ts= pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

In [3]:
tr.shape

(188533, 13)

In [4]:
tr.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [5]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [6]:
df_train = copy.deepcopy(tr)
df_test = copy.deepcopy(ts)

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
print(df_train['engine'].iloc[:150].to_string())

0           172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel
1           252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel
2      320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...
3           420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel
4           208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
5           252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
6          333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
7      355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...
8                        2.7L V6 24V PDI DOHC Twin Turbo
9      425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...
10         312.0HP 3.6L V6 Cylinder Engine Gasoline Fuel
11                   Electric Motor Electric Fuel System
12          420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel
13         333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
14          523.0HP 4.4L 8 Cylinder Engine Gasoline Fuel
15          503.0HP 4.0L 8 Cylinder Engine Gasoline Fuel
16          420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel
17     395.0HP 3.0L Straight 6 

In [9]:
for col in df_train.select_dtypes(include='object'):
    print(col)
    print(df_train[col].nunique())
    print(df_train[col].unique())
    print('-----------------------------------------')

brand
57
['MINI' 'Lincoln' 'Chevrolet' 'Genesis' 'Mercedes-Benz' 'Audi' 'Ford'
 'BMW' 'Tesla' 'Cadillac' 'Land' 'GMC' 'Toyota' 'Hyundai' 'Volvo'
 'Volkswagen' 'Buick' 'Rivian' 'RAM' 'Hummer' 'Alfa' 'INFINITI' 'Jeep'
 'Porsche' 'McLaren' 'Honda' 'Lexus' 'Dodge' 'Nissan' 'Jaguar' 'Acura'
 'Kia' 'Mitsubishi' 'Rolls-Royce' 'Maserati' 'Pontiac' 'Saturn' 'Bentley'
 'Mazda' 'Subaru' 'Ferrari' 'Aston' 'Lamborghini' 'Chrysler' 'Lucid'
 'Lotus' 'Scion' 'smart' 'Karma' 'Plymouth' 'Suzuki' 'FIAT' 'Saab'
 'Bugatti' 'Mercury' 'Polestar' 'Maybach']
-----------------------------------------
model
1897
['Cooper S Base' 'LS V8' 'Silverado 2500 LT' ... 'e-Golf SE'
 'Integra w/A-Spec Tech Package' 'IONIQ Plug-In Hybrid SEL']
-----------------------------------------
fuel_type
7
['Gasoline' 'E85 Flex Fuel' nan 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–'
 'not supported']
-----------------------------------------
engine
1117
['172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel'
 '252.0HP 3.9L 8 Cylinder Engine Gasolin

In [10]:
for col in df_train.select_dtypes(include='object'):
    print(df_train[col].value_counts().sort_values(ascending=False).head(10))
    print('----------------------------------------------------')

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Porsche          10612
Land              9525
Toyota            8850
Lexus             8643
Jeep              6474
Name: count, dtype: int64
----------------------------------------------------
model
F-150 XLT               2945
M3 Base                 2229
Camaro 2SS              1709
M4 Base                 1622
Mustang GT Premium      1526
F-150 Lariat            1410
E-Class E 350 4MATIC    1357
1500 Laramie            1249
911 Carrera S           1219
Corvette Base           1184
Name: count, dtype: int64
----------------------------------------------------
fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64
----------------------------------------------------
engine
355.0HP 5.3L 8 Cylinder Engine Gasoline

In [11]:
df_train.isna().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [12]:
df_train.isna().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [13]:
df_train.drop('id',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 17.3+ MB


# **VIS**

In [15]:
cols_dict = {
      'brand':'brand',
      'model_year':'model_year',
      "accident": "accident",
      "exterior color" : "ext_col",
      "interior color" : "int_col",
}
target='price'

In [16]:
props1 = "background-color: white; color: maroon; font-weight: bold; "
props2 = "background-color: white; color: blue; font-weight: bold; "

In [17]:
for label, col in cols_dict.items():
  t=tr[[col,target]].groupby(col).agg({target: ["max", "min", "mean", "median", "size"]})
  display(t.sort_values(by = col, ascending = True).\
                transpose().\
                style.format(formatter = '{:,.0f}').\
                set_caption(f"\nPrice details by {label}\n").\
                highlight_max(axis=1, props = props1).\
                highlight_min(axis=1, props = props2)
                 )
  print("\n\n\n")

In [18]:
pd.options.display.float_format = '{:.1f}'.format

In [19]:
import re

# **Feature Extraction**

In [20]:
df_train['Horse_power']= df_train['engine'].str.extract(r'(\d+)\.?\d*HP').astype(float)
df_train['Engine_Displacement']= df_train['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
df_train['Cylinder_Count']= df_train['engine'].str.extract(r'(\d+) Cylinder ').astype(float)

df_test['Horse_power']= df_test['engine'].str.extract(r'(\d+)\.?\d*HP').astype(float)
df_test['Engine_Displacement']= df_test['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
df_test['Cylinder_Count']= df_test['engine'].str.extract(r'(\d+) Cylinder ').astype(float)

cylinder_config={'V','Flat','Straight'}
Turbocharger = {'Turbo','Twin Turbo'}
engine_type ={'Gasoline','Electric','Hybrid'}
FUEL_SYSTEMS = {'MPFI', 'GDI', 'PDI', 'TFSI','DOHC', 'SOHC'}

In [21]:
def zoo(word,cylinder_config):
  for i in cylinder_config:
    if i in word:
      return i
  return 'Nan'

In [22]:
df_train['cylinder_config']=df_train['engine'].apply(zoo,args=(cylinder_config,))
df_train['Turbocharger']=df_train['engine'].apply(zoo,args=(Turbocharger,))
df_train['engine_type']=df_train['engine'].apply(zoo,args=(engine_type,))
df_train['FUEL_SYSTEMS']=df_train['engine'].apply(zoo,args=(FUEL_SYSTEMS,))

df_test['cylinder_config']=df_test['engine'].apply(zoo,args=(cylinder_config,))
df_test['Turbocharger']=df_test['engine'].apply(zoo,args=(Turbocharger,))
df_test['engine_type']=df_test['engine'].apply(zoo,args=(engine_type,))
df_test['FUEL_SYSTEMS']=df_test['engine'].apply(zoo,args=(FUEL_SYSTEMS,))

In [23]:
df_train

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Horse_power,Engine_Displacement,Cylinder_Count,cylinder_config,Turbocharger,engine_type,FUEL_SYSTEMS
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0,Nan,Nan,Gasoline,Nan
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0,Nan,Nan,Gasoline,Nan
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0,Nan,Nan,Nan,Nan
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8.0,Nan,Nan,Gasoline,Nan
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0,Nan,Nan,Gasoline,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,6.2,8.0,Nan,Nan,Gasoline,Nan
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,3.0,6.0,V,Nan,Gasoline,Nan
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,4.0,8.0,Nan,Nan,Gasoline,Nan
188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,NaN,3.0,NaN,Nan,Nan,Nan,Nan


In [24]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   brand                188533 non-null  object 
 1   model                188533 non-null  object 
 2   model_year           188533 non-null  int64  
 3   milage               188533 non-null  int64  
 4   fuel_type            183450 non-null  object 
 5   engine               188533 non-null  object 
 6   transmission         188533 non-null  object 
 7   ext_col              188533 non-null  object 
 8   int_col              188533 non-null  object 
 9   accident             186081 non-null  object 
 10  clean_title          167114 non-null  object 
 11  price                188533 non-null  int64  
 12  Horse_power          155274 non-null  float64
 13  Engine_Displacement  174319 non-null  float64
 14  Cylinder_Count       150625 non-null  float64
 15  cylinder_config  

In [25]:
df_train['transmission'].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',


In [26]:
print(df_train['transmission'].iloc[:150].to_string())

0                                    A/T
1                                    A/T
2                                    A/T
3         Transmission w/Dual Shift Mode
4                            7-Speed A/T
5                                    A/T
6                            8-Speed A/T
7                                    A/T
8                     10-Speed Automatic
9         Transmission w/Dual Shift Mode
10                                   A/T
11                           1-Speed A/T
12                           6-Speed A/T
13                                   A/T
14                                   A/T
15                           7-Speed A/T
16                          10-Speed A/T
17                           8-Speed A/T
18                           7-Speed A/T
19                           9-Speed A/T
20                     8-Speed Automatic
21                           8-Speed A/T
22                           6-Speed A/T
23        Transmission w/Dual Shift Mode
24              

In [27]:
df_train['transmission'].value_counts()

transmission
A/T                                                   49904
8-Speed A/T                                           20645
Transmission w/Dual Shift Mode                        19255
6-Speed A/T                                           18044
6-Speed M/T                                           11998
7-Speed A/T                                           11124
Automatic                                             10691
8-Speed Automatic                                      8431
10-Speed A/T                                           8044
9-Speed A/T                                            3866
5-Speed A/T                                            3217
10-Speed Automatic                                     3164
6-Speed Automatic                                      2799
4-Speed A/T                                            2546
5-Speed M/T                                            2409
9-Speed Automatic                                      2325
CVT Transmission           

In [28]:
def extract_transmition (word):
  def get_number ():
    n=re.findall('\d+',word)
    return n[0] if n else ''
  if any(i in word for i in ['AT','A/T','At/Mt','Automatic'] ):
    return ('AT'+str(get_number()))
  elif any(i in word for i in ['CVT'] ):
     return ('CVT'+str(get_number()))
  elif any(i in word for i in ['Manual','M/T'] ):
    return ('MT'+str(get_number()))
  else :
    return 'other'

In [29]:
df_train['new_transmission']=df_train['transmission'].apply(extract_transmition)
df_test['new_transmission']=df_test['transmission'].apply(extract_transmition)

In [30]:
df_train[['new_transmission','transmission']].head(20)

,new_transmission,transmission
0,AT,A/T
1,AT,A/T
2,AT,A/T
3,other,Transmission w/Dual Shift Mode
4,AT7,7-Speed A/T
5,AT,A/T
6,AT8,8-Speed A/T
7,AT,A/T
8,AT10,10-Speed Automatic
9,other,Transmission w/Dual Shift Mode


In [31]:
df_train.drop(['engine','transmission','model'],axis=1,inplace=True)
df_test.drop(['engine','transmission','model'],axis=1,inplace=True)

In [32]:
year = date.today().year
df_train['model_age'] = year - df_train['model_year']
df_test['model_age'] = year - df_test['model_year']

In [33]:
df_train.drop('model_year',axis=1,inplace=True)
df_test.drop('model_year',axis=1,inplace=True)

In [34]:
df_train.head()

,brand,milage,fuel_type,ext_col,int_col,accident,clean_title,price,Horse_power,Engine_Displacement,Cylinder_Count,cylinder_config,Turbocharger,engine_type,FUEL_SYSTEMS,new_transmission,model_age
0,MINI,213000,Gasoline,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0,Nan,Nan,Gasoline,Nan,AT,17
1,Lincoln,143250,Gasoline,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0,Nan,Nan,Gasoline,Nan,AT,22
2,Chevrolet,136731,E85 Flex Fuel,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0,Nan,Nan,Nan,Nan,AT,22
3,Genesis,19500,Gasoline,Black,Black,None reported,Yes,45000,420.0,5.0,8.0,Nan,Nan,Gasoline,Nan,other,7
4,Mercedes-Benz,7388,Gasoline,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0,Nan,Nan,Gasoline,Nan,AT7,3


In [35]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   brand                188533 non-null  object 
 1   milage               188533 non-null  int64  
 2   fuel_type            183450 non-null  object 
 3   ext_col              188533 non-null  object 
 4   int_col              188533 non-null  object 
 5   accident             186081 non-null  object 
 6   clean_title          167114 non-null  object 
 7   price                188533 non-null  int64  
 8   Horse_power          155274 non-null  float64
 9   Engine_Displacement  174319 non-null  float64
 10  Cylinder_Count       150625 non-null  float64
 11  cylinder_config      188533 non-null  object 
 12  Turbocharger         188533 non-null  object 
 13  engine_type          188533 non-null  object 
 14  FUEL_SYSTEMS         188533 non-null  object 
 15  new_transmission 

In [36]:
df_train.isna().sum()

brand                      0
milage                     0
fuel_type               5083
ext_col                    0
int_col                    0
accident                2452
clean_title            21419
price                      0
Horse_power            33259
Engine_Displacement    14214
Cylinder_Count         37908
cylinder_config            0
Turbocharger               0
engine_type                0
FUEL_SYSTEMS               0
new_transmission           0
model_age                  0
dtype: int64

In [37]:
y=df_train['price']
df_train=df_train.drop('price',axis=1)

In [38]:
x_train,x_test,y_train,y_test=train_test_split(df_train,y,test_size=.2,random_state=42)

In [39]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   brand                188533 non-null  object 
 1   milage               188533 non-null  int64  
 2   fuel_type            183450 non-null  object 
 3   ext_col              188533 non-null  object 
 4   int_col              188533 non-null  object 
 5   accident             186081 non-null  object 
 6   clean_title          167114 non-null  object 
 7   Horse_power          155274 non-null  float64
 8   Engine_Displacement  174319 non-null  float64
 9   Cylinder_Count       150625 non-null  float64
 10  cylinder_config      188533 non-null  object 
 11  Turbocharger         188533 non-null  object 
 12  engine_type          188533 non-null  object 
 13  FUEL_SYSTEMS         188533 non-null  object 
 14  new_transmission     188533 non-null  object 
 15  model_age        

# **PipeLine**

In [40]:
num_col = df_train.select_dtypes(include='number').columns
cat_col = df_train.select_dtypes(include='object').columns

In [41]:
# # num_pipe=Pipeline(steps=[
#       # ('imputer',SimpleImputer(strategy='mean')),
#       # ('scaler',StandardScaler() )  ])
# cat_pipe=Pipeline(steps=[
#       ('imputer',SimpleImputer(strategy='most_frequent')),
#       ('encoder',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1))
#                  ] )


In [42]:
# df_transformer=ColumnTransformer(
#             transformers=[
#                   # ('num', num_pipe, num_col),
#                   ('cat', cat_pipe, cat_col)
#                  ] )

In [43]:
# final_pipe= Pipeline(steps=[
#     ('prepcorcessor',df_transformer)
# ])

In [44]:
# x_train_done = final_pipe.fit_transform(x_train)
# x_test_done = final_pipe.transform(x_test)

In [45]:
num_imputer= SimpleImputer(strategy='mean')
cat_imputer= SimpleImputer(strategy='most_frequent')
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
scaler = StandardScaler()
for col in num_col:
  x_train[col] = num_imputer.fit_transform(x_train[[col]])
  x_test[col] = num_imputer.transform(x_test[[col]])

  df_test[col] = num_imputer.transform(df_test[[col]])

  x_train[col] = scaler.fit_transform(x_train[[col]])
  x_test[col] = scaler.transform(x_test[[col]])
  df_test[col] = scaler.transform(df_test[[col]])
for col in cat_col:
  x_train[[col]] = cat_imputer.fit_transform(x_train[col].values.reshape(-1,1))
  x_test[[col]] = cat_imputer.transform(x_test[col].values.reshape(-1,1))

  df_test[[col]] = cat_imputer.transform(df_test[col].values.reshape(-1,1))

  x_train[col] = encoder.fit_transform(x_train[[col]])
  x_test[col] = encoder.transform(x_test[[col]])

  df_test[col] = encoder.transform(df_test[[col]])


In [46]:
df_train.head()

,brand,milage,fuel_type,ext_col,int_col,accident,clean_title,Horse_power,Engine_Displacement,Cylinder_Count,cylinder_config,Turbocharger,engine_type,FUEL_SYSTEMS,new_transmission,model_age
0,MINI,213000,Gasoline,Yellow,Gray,None reported,Yes,172.0,1.6,4.0,Nan,Nan,Gasoline,Nan,AT,17
1,Lincoln,143250,Gasoline,Silver,Beige,At least 1 accident or damage reported,Yes,252.0,3.9,8.0,Nan,Nan,Gasoline,Nan,AT,22
2,Chevrolet,136731,E85 Flex Fuel,Blue,Gray,None reported,Yes,320.0,5.3,8.0,Nan,Nan,Nan,Nan,AT,22
3,Genesis,19500,Gasoline,Black,Black,None reported,Yes,420.0,5.0,8.0,Nan,Nan,Gasoline,Nan,other,7
4,Mercedes-Benz,7388,Gasoline,Black,Beige,None reported,Yes,208.0,2.0,4.0,Nan,Nan,Gasoline,Nan,AT7,3


In [47]:
df_train['clean_title']

0         Yes
1         Yes
2         Yes
3         Yes
4         Yes
         ... 
188528    Yes
188529    Yes
188530    Yes
188531    NaN
188532    Yes
Name: clean_title, Length: 188533, dtype: object

In [48]:
zob=pd.DataFrame(x_train,columns=df_train.columns)
zob

,brand,milage,fuel_type,ext_col,int_col,accident,clean_title,Horse_power,Engine_Displacement,Cylinder_Count,cylinder_config,Turbocharger,engine_type,FUEL_SYSTEMS,new_transmission,model_age
184031,36.0,-0.1,1.0,128.0,14.0,0.0,0.0,-1.3,-1.4,-1.7,1.0,0.0,3.0,3.0,16.0,-0.2
173831,39.0,2.4,2.0,263.0,71.0,0.0,0.0,-1.0,0.1,-0.3,3.0,0.0,1.0,3.0,0.0,2.3
183819,14.0,0.5,2.0,128.0,71.0,0.0,0.0,-0.2,-0.8,-0.3,3.0,0.0,1.0,3.0,2.0,-0.7
85525,19.0,-1.2,2.0,29.0,14.0,1.0,0.0,-0.6,-1.4,-1.7,1.0,0.0,1.0,3.0,13.0,-1.3
41872,11.0,-1.2,2.0,234.0,14.0,1.0,0.0,3.6,1.8,1.2,1.0,0.0,1.0,3.0,6.0,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,9.0,0.5,2.0,234.0,10.0,0.0,0.0,0.1,1.5,1.2,1.0,0.0,1.0,3.0,13.0,1.6
103694,4.0,-0.2,2.0,29.0,14.0,1.0,0.0,-0.2,-0.6,-0.3,2.0,0.0,1.0,3.0,7.0,-0.6
131932,43.0,-0.7,2.0,29.0,14.0,1.0,0.0,0.7,-0.6,-0.3,0.0,0.0,1.0,3.0,8.0,-0.9
146867,36.0,2.4,2.0,29.0,14.0,1.0,0.0,-0.7,-0.2,-0.3,3.0,0.0,1.0,3.0,7.0,0.9


In [49]:
isolation_forest = IsolationForest(contamination=0.024, random_state=42)
x_train_labels = isolation_forest.fit_predict(x_train) # 1 indicate normal, -1--> outlier (anomaly)
normal_bool = x_train_labels != -1 
x_train=x_train[normal_bool]
y_train=y_train[normal_bool]

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [50]:
(1-len(x_train)/len(x_train_labels))*100

2.4001166907562377

In [51]:
import xgboost as xgb
from lightgbm import LGBMRegressor

In [52]:
xgb=xgb.XGBRegressor(tree_method ="hist",
                          device               = "cuda" ,
                          objective             = "reg:squarederror",
                          eval_metric           = "rmse",
                          random_state          = 42,
                          colsample_bytree      = 0.45,
                          learning_rate       = 0.025,
                          max_depth          = 7,
                          n_estimators          = 3000,
                          reg_alpha          = 0.001,
                        reg_lambda            = 0.001,
                          min_child_weight      = 18,
                          verbosity = 0,
                          enable_categorical    = True,
)
lgb_params= {
        'n_epochs': 1000,
        'learning_rate': 0.01,
        'colsample_bytree': 0.55,
        'bagging_fraction': 0.8,
        'num_leaves': 3072,
        'min_child_samples': 12,
        'reg_lambda': 64, # not 128
        'max_bin': 255,
        'max_depth': 10,
        'reg_alpha': 0,
        'verbose': -1,
    }

In [53]:
lgb_reg = LGBMRegressor(**lgb_params,loss_function='RMSE',verbosity=-1)
lgb_reg.fit(x_train,y_train)
xgb.fit(x_train,y_train)
y_pred1=xgb.predict(x_test)
y_pred= lgb_reg.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,y_pred) ))
np.sqrt(mean_squared_error(y_test,y_pred1) )

69592.87081646528


69100.10041921548

In [60]:
vtr=VotingRegressor(estimators=[('xgboost',xgb),('lightbgm',lgb_reg)])
vtr.fit(x_train,y_train)
y_zp=vtr.predict(x_test)
mean_squared_error(y_test,y_zp,squared=False)

68405.0197907535

In [54]:
print(r2_score(y_test,y_pred))
r2_score(y_test,y_pred1)

0.1291045923905203


0.1413941304339712

In [61]:
y_p1= xgb.predict(df_test)
y_p2 = lgb_reg.predict(df_test)
y_p3 = vtr.predict(df_test)
# print(np.sqrt(mean_squared_error(y_zz,y_p1) ))
# print(np.sqrt(mean_squared_error(y_zz,y_p2) ))

In [62]:
submession = pd.DataFrame({
    'id':ts['id'].values,
    'price':y_p3

})

In [63]:
submession.head()

,id,price
0,188533,21903.1
1,188534,76869.9
2,188535,55443.1
3,188536,42364.0
4,188537,32491.2


In [64]:
submession.to_csv('submession.csv', index=False)